In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-09-12 17:24 ajna         INFO     Configuração de log efetuada


2025-09-12 17:24:18,419 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204')" # Redex da Localfrio/Movecta e Redex da Santos Brasil Clia
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=3)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
"""

# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 22 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 22 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL para saida_redex (alias c)
sql_saida_redex = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, direcao
FROM apirecintos_acessosveiculo
WHERE
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'S'
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND numeroConteiner !=''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento

In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento
0,12217222,BSG3J05,EGSU3507783,8931318,2025-09-09 17:38:53,C,E,21425093000176,31347301895,MOACIR DOMINGUES FABBRIS,,
1,12217223,DVS2C47,MRKU3696302,8931318,2025-09-09 17:40:36,C,E,21425093000176,31328693848,ANDERSON BUENO CAMPOS,,
2,12217226,OZD7B52,CAAU9836114,8931318,2025-09-09 17:43:03,C,E,21425093000176,21864622814,KLEBER CUCO DA SILVA,,
3,12217225,BWB7761,TXGU8418253,8931318,2025-09-09 17:44:19,C,E,21425093000176,29372447854,NELSON RAMOS ALVES,,
4,12217227,BTS6G42,MRKU0263314,8931318,2025-09-09 17:45:33,C,E,12014047000193,27066259882,LUCIANO RODRIGO SANTOS DE LIMA,,
...,...,...,...,...,...,...,...,...,...,...,...,...
10354,12296698,MJO9I50,TLLU5201283,8931404,2025-09-11 23:32:43,C,E,42548097000111,16960826845,SILVIO APARECIDO DA SILVA,,
10355,12296721,AAW2B18,ONEU9398611,8931404,2025-09-11 23:39:33,C,E,21425093000176,37339188860,RONALDO LUIZ BARROS SOUSA,,
10356,12296722,RNR3I42,GESU1213432,8931404,2025-09-11 23:39:40,C,E,31962717000148,29710532880,CRISTIANO WILLIANS VIEIRA BRAS,35250908299558000121550010000002051383403952,
10357,12296741,CUA6485,FANU1986938,8931404,2025-09-11 23:48:18,C,E,45050663000159,21569023808,MARCELO DA SILVA MOURA,,


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,direcao
0,12230274,SDT6A70,EGHU3345694,8932775,2025-09-09 17:24:47,S
1,12230001,TLV5G33,CGMU5629133,8932761,2025-09-09 17:29:14,S
2,12230005,STQ0E43,MNBU4403760,8932761,2025-09-09 17:34:40,S
3,12229421,MQJ3J01,HAMU4572800,8932722,2025-09-09 17:43:54,S
4,12230008,GFN4I21,MNBU0049088,8932761,2025-09-09 17:56:45,S
...,...,...,...,...,...,...
1020,12299613,BXF4E94,MEDU5888617,8932797,2025-09-11 23:36:02,S
1021,12299612,DPC9B40,MEDU5443914,8932797,2025-09-11 23:36:39,S
1022,12297971,SWU1E80,SEGU9994172,8932761,2025-09-11 23:37:29,S
1023,12299210,MIH1F29,TCKU1854406,8932796,2025-09-11 23:42:26,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placa', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                     df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
# df_saidas_entradas

In [16]:
#!pip install tqdm
from tqdm.notebook import tqdm

def buscar_id_fs_files(collection, numero_conteiner, data_ocorrencia):
    filtro = {
        'metadata.numeroinformado': numero_conteiner,
        'metadata.dataescaneamento': {
            '$gte': data_ocorrencia - timedelta(hours=3),
            '$lte': data_ocorrencia + timedelta(hours=6)
        }
    }
    resultado = collection.find_one(filtro, {'_id': 1})
    if resultado:
        return resultado['_id']
    return None

# Iterar sobre as linhas do DataFrame e aplicar busca
def mapear_ids_mongo(collection, df):
    ids_fs_files = []
    cont_sucesso = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Buscando ids no MongoDB"):
        numero = row['numeroConteiner']
        data = row['dataHoraOcorrencia_E_Operador']
        id_fs = buscar_id_fs_files(collection, numero, data)
        ids_fs_files.append(id_fs)
        cont_sucesso += 1 if id_fs else 0
    df['fs_files_id'] = ids_fs_files
    print(f'Registros: {len(df)}, imagens: {cont_sucesso}')

In [17]:
mapear_ids_mongo(db['fs.files'], df_saidas_entradas)

Buscando ids no MongoDB:   0%|          | 0/741 [00:00<?, ?it/s]

Registros: 741, imagens: 204


In [18]:
# df_saidas_entradas.transitTime.describe()
df_saidas_entradas.rename(columns={'placa': 'placa', 'numeroConteiner': 'contêiner',
                         'codigoRecinto_S_REDEX': 'Redex', 'dataHoraOcorrencia_S_REDEX': 'Saída REDEX',
                         'codigoRecinto_E_Operador': 'Operador', 'dataHoraOcorrencia_E_Operador': 'Entrada Operador',
                         'transitTime': 'transitTime', 'fs_files_id': '_id'}, inplace=True)
df_saidas_entradas.drop(['operacao', 'direcao_E_Operador', 'listaNfe', 'numeroConhecimento', 'direcao_S_REDEX'], axis=1)
df_saidas_entradas

,id_E_Operador,placa,contêiner,Operador,Entrada Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,id_S_REDEX,Redex,Saída REDEX,direcao_S_REDEX,transitTime,_id
0,12221977,IBL8235,TXGU7093174,8931356,2025-09-09 19:19:19,C,E,58130089000786,27367033832,ADRIANO FIRMINO DA SILVA,35250871320915005000550030000596861581684783,,12230918,8932796,2025-09-09 18:41:54,S,0.623611,68c16eef035403c27a3951fb
1,12222071,SDT6A70,EGHU3345694,8931356,2025-09-09 19:43:40,C,E,01984600000102,30606725806,MARCOS APARECIDO SOARES,31250923117229000106550010002202181168787658,,12230274,8932775,2025-09-09 17:24:47,S,2.314722,68c16eee035403c27a3951a0
2,12222375,JLY8F87,TEMU8227505,8931356,2025-09-09 21:10:08,C,E,58130089000786,25358504869,LAERCIO MELO DE LIRA,31250929627430000209550010000789691711922462,,12230950,8932796,2025-09-09 20:09:48,S,1.005556,68c16f52035403c27a39553a
3,12222384,MTR6E99,ONEU0064906,8931356,2025-09-09 21:11:40,C,E,58130089000786,33962306803,TAMER BARBOSA,31250929627430000209550010000789691711922462,,12230649,8932796,2025-09-09 20:12:41,S,0.983056,68c16f52035403c27a395525
4,12222389,NRZ0E47,BEAU5299503,8931356,2025-09-09 21:11:41,C,E,58130089000786,30845535803,RAPHAEL PERES MELO,31250929627430000209550010000789691711922462,,12230945,8932796,2025-09-09 20:01:28,S,1.170278,68c16f51035403c27a3954d6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736,12296433,SWO3C33,HLXU1061105,8931404,2025-09-11 22:12:22,C,E,51641405000140,28481410934,JORGE BOEIRA DOS SANTOS,35250945736992000158550000000240521547005889,,12262436,8932722,2025-09-10 22:37:54,S,23.574444,None
737,12296598,NTO9D51,HAMU3316213,8931404,2025-09-11 22:58:02,C,E,58130089000786,39914451829,GLEIDSON JUNIOR SANTOS,,,12299824,8932798,2025-09-11 21:27:01,S,1.516944,None
738,12296606,FQN3763,HAMU2437416,8931404,2025-09-11 22:59:08,C,E,58130089000786,37889811802,MIGUEL RUIZ DOS SANTOS,,,12299821,8932798,2025-09-11 20:51:12,S,2.132222,None
739,12296617,ASX1H52,HAMU3055906,8931404,2025-09-11 23:02:20,C,E,58130089000786,00113214588,JORGE CASCIO SOUZA SANTOS SANT,,,12299705,8932798,2025-09-11 21:22:45,S,1.659722,None


In [19]:
from IPython.display import display, HTML

def get_outliers(df):
    mean = df['transitTime'].mean()
    std = df['transitTime'].std()
    outliers = df[(df['transitTime'] < mean - 1 * std) | (df['transitTime'] > mean + 1 * std)]
    return outliers

def generate_html(_id):
    url = f"https://ajna1.rfoc.srf/virasana/file?_id={_id}"
    img = f"https://ajna1.rfoc.srf/virasana/imagens_cmap?_id={_id}"
    return f'<a href="{url}"><img src="{img}" width="200"></a>'
    
outliers = get_outliers(df_saidas_entradas)[['placa', 'contêiner', 'Redex', 'Saída REDEX',
                                  'Operador', 'Entrada Operador', 'transitTime', '_id']]

outliers['html'] = outliers['_id'].apply(generate_html)
display(HTML(outliers.to_html(escape=False)))

,placa,contêiner,Redex,Saída REDEX,Operador,Entrada Operador,transitTime,_id,html
46,GEU6F66,SELU4393340,8932761,2025-09-10 07:41:00,8931356,2025-09-10 02:54:24,-4.776667,None,
327,AXN2C67,HLXU8202041,8932796,2025-09-10 16:22:31,8931356,2025-09-11 13:58:19,21.596667,None,
328,AXN2C67,HLXU8202041,8932796,2025-09-10 16:47:45,8931356,2025-09-11 13:58:19,21.176111,None,
377,FOO4D14,FANU1401220,8932775,2025-09-11 09:13:17,8931356,2025-09-11 18:11:24,8.968611,None,
385,FSP6G23,TGBU9609122,8932775,2025-09-10 23:22:12,8931356,2025-09-11 20:55:31,21.555278,None,
463,FMK9D73,MEDU3539391,8932722,2025-09-09 21:57:43,8931359,2025-09-10 16:45:45,18.800556,None,
469,SJM2C43,CMAU3610526,8932722,2025-09-09 19:23:28,8931359,2025-09-10 22:07:08,26.727778,None,
640,OVS9B36,MSNU7884747,8932761,2025-09-11 18:06:35,8931359,2025-09-11 22:05:54,3.988611,68c419928e8f95cb27befe59,
688,CUA3G00,HLBU2354950,8932796,2025-09-10 09:30:40,8931404,2025-09-10 18:59:41,9.483611,None,
713,EQW0A50,GLDU9541584,8932796,2025-09-10 12:44:37,8931404,2025-09-11 01:17:54,12.554722,None,


In [20]:
# Adicionar células clicáveis que levem ao Ajna
df_saidas_entradas['contêiner'] = df_saidas_entradas['contêiner'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/bhadrasana2/consulta_container?numerolote={x}", "{x}")'
)
df_saidas_entradas['_id'] = df_saidas_entradas['_id'].apply(
    lambda x: f'=HYPERLINK("https://ajna1.rfoc.srf/virasana/file?_id={x}", "{x}")'
)

# df_saidas_entradas

In [21]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx', index=False)

In [22]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placa', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 3) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placa', 'numeroConteiner'], right_on=['placa', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 3) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) &
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

SyntaxError: invalid syntax (1004912324.py, line 26)